In [19]:
import pandas as pd
import numpy as np
from timeit import default_timer as timer
from functools import wraps
import ipywidgets as widgets

def print_timing(f):
    @wraps(f)
    def inner(*args, **kwargs):
        t0 = timer()
        try:
            return f(*args, **kwargs)
        finally:
            print(f"\n{f.__name__} finished in {1e3*(timer()-t0):.1f} ms")
    return inner

In [20]:


from fuzzywuzzy import fuzz as fuzzywuzzy_fuzz, process as fuzzywuzzy_process
from rapidfuzz import fuzz as rapidfuzz_fuzz, process as rapidfuzz_process
import Levenshtein

In [21]:


#   You can download this dataset here:
#   https://geoportal.statistics.gov.uk/datasets/a6c138d17ac54532b0ca8ee693922f10_0

df = pd.read_csv("Index_of_Place_Names_in_Great_Britain_(July_2016).csv")
cities = df["place15nm"].unique()

@print_timing
def interact_cities(input_text):
    if not input_text: return
    for city, score, _ in rapidfuzz_process.extract(input_text, cities, limit=10):
        print(f"{city:60}{score:3.2f}")

widgets.interact(interact_cities, input_text="");



interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

In [22]:
fuzzywuzzy_fuzz.ratio("Tokyo and Osaka", "Tokio and Osaka")

93

In [23]:


words = ["knight", "knuth", "nigh", "ignite", "knighthood", "knead", "the"]
fuzzywuzzy_process.extract("knigth", words)



[('knight', 83), ('nigh', 80), ('knighthood', 75), ('knuth', 73), ('the', 72)]

Search Algos and How they work with

In [24]:
fuzzywuzzy_fuzz.ratio("mancesther", "manchester")

90

How do you measure similarity of two strings a, b?

    Number of editing operations to get from a to b
        Replace (Hamming)
        Replace, insert, delete (Levenshtein)
        Replace, insert, delete, transpose (Levenshtein-Damerau)
        ...
    Longest common substring
    Common word count
    Letter frequency



**Hamming Distance Explanation**

Hamming distance is the number of letters one has to replace to get from string a to string b.

    d("cat", "hat") = 1, replace 1st letter (c -> h)
    d("cat", "lag") = 2, replace 1st letter (c -> l) and 3rd letter (t -> g)
    d("cat", "cats") is conventionally not defined (it cannot be achieved by replacing letters)


**Hamming distance is used in study of error-correcting codes (encoding schemes that are resilient to some number of corrupt bits). Its major limitation in that it doesn't handle misaligned sequences well.**

'''
d("Hamming distance",
  "Hammingdistance ") = 9
          xxxxxxxxx
'''

In [30]:


def hamming_distance(a: str, b: str):
    if len(a) == len(b):
        return sum(1 for a_i, b_i in zip(a, b) if a_i != b_i)
    else:
        return float("inf")  # cannot get different length


@print_timing
def interact_cities_hamming(input_text):
    if not input_text: return
    for score, city in list(sorted((hamming_distance(input_text, s), s) for s in cities))[:10]:
        print(f"{city:60}{score:3.2f}")

widgets.interact(interact_cities_hamming, input_text="");



interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

-------



## Levenshtein Distance with DP(Wagner-Fischer)
[Wagner Fisher algorithm - Wiki](https://en.wikipedia.org/wiki/Wagner%E2%80%93Fischer_algorithm)

Runtime - O(n^2)

Compared to optimized C implementation, below levenshtein_distance() is 100-500x slower.

In [31]:


def levenshtein_distance(a: str, b: str, verbose: bool = False) -> int:
    m, n = len(a), len(b)
    d = np.zeros((m+1, n+1), dtype=int)  # d[i,j] = levenshtein_distance(a[:i], b[:j])
        
    # when the other string is empty, distance is length of non-empty string
    for i in range(m+1): d[i, 0] = i
    for i in range(n+1): d[0, i] = i
    
    for j in range(1, n+1):
        for i in range(1, m+1):
            cost = 1 if a[i-1] != b[j-1] else 0
            d[i, j] = min(d[i-1, j-1] + cost,   # substitute         ↘
                          d[i, j-1]   + 1,      # delete from B      →
                          d[i-1, j]   + 1)      # insert into B      ↓
    
    if verbose: print(d)
    return d[m, n]



In [32]:
d = levenshtein_distance("cat", "wildcat", verbose=True)
d

[[0 1 2 3 4 5 6 7]
 [1 1 2 3 4 4 5 6]
 [2 2 2 3 4 5 4 5]
 [3 3 3 3 4 5 5 4]]


4

In [33]:
# optimized C implementation, https://pypi.org/project/python-Levenshtein

%timeit Levenshtein.distance("cat", "rat")
%timeit Levenshtein.distance("knight", "knigth")
%timeit Levenshtein.distance("unimaginable", "imagination")



79.1 ns ± 0.349 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
118 ns ± 1.07 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
254 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [34]:
%timeit levenshtein_distance("cat", "rat")
%timeit levenshtein_distance("knight", "knigth")
%timeit levenshtein_distance("unimaginable", "imagination")

11.4 µs ± 92.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
39.8 µs ± 262 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
139 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [35]:
# CITIES EXAMPLE
@print_timing
def interact_cities_levenshtein(input_text):
    n = len(input_text)
    def rank(s):
        m = len(s)
        if abs(m - n) > 5:  # 🔧 cheating a bit - we'll skip words which would have
            return 0        #                     a large distance anyway
        elif m == n:
            dist = hamming_distance(input_text, s)  # 🔧 not cheating, gives the same results
        else:
            dist = levenshtein_distance(input_text, s)
        return 1.0 - dist/max(n, m)
        
    if not input_text: return
    for score, city in list(sorted(((rank(s), s) for s in cities), reverse=True))[:10]:
        print(f"{city:60}{100*score:3.2f}")

widgets.interact(interact_cities_levenshtein, input_text="");

interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

### Vectorization 

Our Wagner-Fischer in Python is close, but not quite there for real-time querying. One possible thing to do here would be to optimize the function as it is, eg. using Cython or Numba.

However, there is a different way — we can compute many distances at once, using the exact same logic. Indeed, for all strings of the same length, the algorithm does exactly the same steps, just the values in the table are different.

Instead of one 2D table, we can have a "3D table", really just many 2D tables stacked together. Then d[i,j] will not be just one number, but a vector holding the results for all processed strings at once.

This should be significantly faster, because the operations we need (sum, comparison, minimum) can be executed very efficiently for vectors using eg. NumPy. The Python function itself will be as slow as ever, but we'll only need to call it once, not thousands of times. Note that this doesn't change the asymptotic running time at all, it's just about the efficiency of Python vs. native code.

In [36]:


def levenshtein_distance_vec(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    m, n, k = len(a), len(b), b.shape[1]
    d = np.zeros((m+1, n+1, k), dtype=np.uint16)  # d[i,j] = levenshtein_distance(a[:i], b[:j])

    # when the other string is empty, distance is length of non-empty string
    for i in range(m+1): d[i, 0] = i
    for i in range(n+1): d[0, i] = i
    
    for j in range(1, n+1):
        for i in range(1, m+1):
            cost = a[i-1] != b[j-1]
            d[i, j] = np.min([d[i-1, j-1] + cost,        # substitute         ↘
                              d[i, j-1]   + 1,           # delete from B      →
                              d[i-1, j]   + 1], axis=0)  # insert into B      ↓

    return d[m, n]



In [37]:


# convert input strings to NumPy matrices
len_to_cities = {}
for w in cities: len_to_cities.setdefault(len(w), []).append(w)
len_to_mat = {n: np.asarray([[ord(c) for c in w] for w in ws], dtype=np.uint16).T
              for n, ws in len_to_cities.items()}

@print_timing
def interact_cities_levenshtein_vec(input_text):
    a_vec = np.asarray([ord(c) for c in input_text])
    n = len(input_text)
    xs, ys = [], []
    
    for m, b_mat in len_to_mat.items():
        if abs(m - n) > 5: continue  # 🔧 again, skipping strings with obvious large distance
        dist = levenshtein_distance_vec(a_vec, b_mat)
        scores = 1.0 - dist/max(n, m)
        xs.extend(len_to_cities[m])
        ys.extend(scores) 
    
    if not input_text: return
    for score, city in list(sorted(zip(ys, xs), reverse=True))[:10]:
        print(f"{city:60}{score:3.2f}")

widgets.interact(interact_cities_levenshtein_vec, input_text="");



interactive(children=(Text(value='', description='input_text'), Output()), _dom_classes=('widget-interact',))

### Levenshtein in C

In [38]:
a = "londen"
a_vec = np.asarray([ord(c) for c in a])

print("levenshtein_distance")
%timeit -r 1 -n 1 [levenshtein_distance(a, b) for b in cities]
print("\nlevenshtein_distance_vec")
%timeit [levenshtein_distance_vec(a_vec, b_mat) for b_mat in len_to_mat.values()]
print("\nC implementation")
%timeit [Levenshtein.distance(a, b) for b in cities]

levenshtein_distance
4.67 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

levenshtein_distance_vec
155 ms ± 6.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

C implementation
14.2 ms ± 85.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
